In [1]:
import sys
sys.path.append("../Modules/")
sys.path.append("../pyFiles/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np

# from configobj import ConfigObj
# from configobj import flatten_errors
# from validate import Validator
    
import logging

# from analysisParameters import loadParameters
# from loadDataE1 import loadSessionDict

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import os

init_notebook_mode(connected=True)


In [2]:
# sessionDict = projectTrajectories(sessionDict)
import matplotlib.pyplot as plt
%matplotlib notebook

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]

In [4]:
list(sessionDict.keys())

['subID',
 'trialInfo',
 'expConfig',
 'rawExpUnity',
 'rawExpGaze',
 'processedExp',
 'rawCalibUnity',
 'rawCalibGaze',
 'processedCalib',
 'analysisParameters']

In [5]:
processedCalib = sessionDict['processedCalib']
list(processedCalib)

[('azimuthWidth', ''),
 ('base_data', ''),
 ('blockNumber', ''),
 ('camera', 'R0C0'),
 ('camera', 'R0C1'),
 ('camera', 'R0C2'),
 ('camera', 'R0C3'),
 ('camera', 'R1C0'),
 ('camera', 'R1C1'),
 ('camera', 'R1C2'),
 ('camera', 'R1C3'),
 ('camera', 'R2C0'),
 ('camera', 'R2C1'),
 ('camera', 'R2C2'),
 ('camera', 'R2C3'),
 ('camera', 'R3C0'),
 ('camera', 'R3C1'),
 ('camera', 'R3C2'),
 ('camera', 'R3C3'),
 ('cameraPos', 'x'),
 ('cameraPos', 'y'),
 ('cameraPos', 'z'),
 ('cameraRot', 'x'),
 ('cameraRot', 'y'),
 ('cameraRot', 'z'),
 ('confidence', ''),
 ('currentTargetName', ''),
 ('elevationHeight', ''),
 ('eye-center0-3d', 'x'),
 ('eye-center0-3d', 'y'),
 ('eye-center0-3d', 'z'),
 ('eye-center1-3d', 'x'),
 ('eye-center1-3d', 'y'),
 ('eye-center1-3d', 'z'),
 ('fixationTime', ''),
 ('frameTime', ''),
 ('gaze-normal0', 'x'),
 ('gaze-normal0', 'y'),
 ('gaze-normal0', 'z'),
 ('gaze-normal1', 'x'),
 ('gaze-normal1', 'y'),
 ('gaze-normal1', 'z'),
 ('gaze-point-3d', 'x'),
 ('gaze-point-3d', 'y'),
 ('ga

In [6]:
processedCalib['worldVidTimeSec'] = np.array(processedCalib['pupilTimestamp'] - processedCalib['pupilTimestamp'].iloc[0])
processedCalib['worldVidTimeSec'].tail(10)

2965    26.072226
2966    26.076210
2967    26.076532
2968    26.080378
2969    26.084560
2970    26.087518
2971    26.092663
2972    26.096482
2973    26.098138
2974    26.102518
Name: worldVidTimeSec, dtype: float64

### Calc cyc to ball direction

In [7]:

cycToBallVec = np.array(sessionDict['processedCalib']['targetPos'] - sessionDict['processedCalib']['cameraPos'],dtype= np.float64 )
cycToBallDir = np.array([np.divide(XYZ,np.linalg.norm(XYZ)) for XYZ in cycToBallVec],dtype= np.float64)

sessionDict['processedCalib']['ball_az'] = np.rad2deg(np.arctan(cycToBallDir[:,0]/cycToBallDir[:,2]))
sessionDict['processedCalib']['ball_el'] = np.rad2deg(np.arctan(cycToBallDir[:,1]/cycToBallDir[:,2]))
                                                         

### Recalc GIWDir

In [8]:
def calcGIW_ByRow(rowIn):
    
    # Grab gransformation matrix
    headTransform_4x4 = np.reshape(rowIn['camera'].values,[4,4])

    # Grab cyc EIH direction
    cycEyeInHead_XYZ = rowIn['gaze_normal2']

    # Add a 1 to convert to homogeneous coordinates
    cycEyeInHead_XYZW = np.hstack( [cycEyeInHead_XYZ,1])

    # Take the dot product!
    cycGIWVec_XYZW = np.dot( headTransform_4x4,cycEyeInHead_XYZW)

    # # Now, convert into a unit direction vector from the cyclopean eye in world coordinates
    # # Also, we can discard the w term
    cycGIWDir_XYZ = (cycGIWVec_XYZW[0:3]-rowIn["cameraPos"]) / np.linalg.norm((cycGIWVec_XYZW[0:3]-rowIn["cameraPos"]))

    # # You must return as a list or a tuple
    # #return list(cycGIWDir_XYZ)
    return {('cycGIWDir','x'): cycGIWDir_XYZ[0],('cycGIWDir','y'): cycGIWDir_XYZ[1],('cycGIWDir','z'): cycGIWDir_XYZ[2]}

dictListOut = sessionDict['processedCalib'].apply(lambda row: calcGIW_ByRow(row),axis=1)
dfOut = pd.DataFrame.from_records(dictListOut)
dfOut.columns = pd.MultiIndex.from_tuples(dfOut.columns)

if 'cycGIWDir' in sessionDict['processedCalib'].columns:
    sessionDict['processedCalib'].drop("cycGIWDir", axis=1, level=0,inplace=True)
            
sessionDict['processedCalib'] = sessionDict['processedCalib'].join(dfOut)

In [9]:

proc = sessionDict['processedCalib']
sessionDict['processedCalib']['cycGIW_az'] = np.rad2deg(np.arctan(proc[('cycGIWDir','x')]/proc[('cycGIWDir','z')]))
sessionDict['processedCalib']['cycGIW_el'] = np.rad2deg(np.arctan(proc[('cycGIWDir','y')]/proc[('cycGIWDir','z')]))
                                                         

## Plot calib assessment using world frame spherical (GIW)

In [10]:
df2 = sessionDict['processedCalib']

yLim=[-50,50]
width=800
height=600

startTimeS = df2['worldVidTimeSec'].iloc[0]
endTimeS = df2['worldVidTimeSec'].iloc[-2]

# I googled for, "python find first larger than"
startIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  startTimeS)
endIndex = next(idx for idx, val in enumerate(df2['worldVidTimeSec']) if val >  endTimeS)
print('Plotting from index: ' + str(startIndex) + ' to ' + str(endIndex))

colors_idx = ['rgb(0,204,204)','rgb(128,128,128)','rgb(204,0,0)','rgb(102,0,204)']

# from plotly.subplots import make_subplots
# fig = make_subplots(rows=2, cols=1,shared_xaxes=True)

traces = []

###

bEih_el = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_el'][startIndex:endIndex],
    name = 'ball_el',
    marker_color = colors_idx[2],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)

bEih_az = go.Scattergl(
    x=df2['worldVidTimeSec'][startIndex:endIndex],
    y=df2['ball_az'][startIndex:endIndex],
    name = 'ball_az',
    marker_color = colors_idx[3],
    mode='markers',
    marker_size = 8,
    opacity = 0.5)


traces.append(bEih_el)
traces.append(bEih_az)


eih_el = go.Scattergl(
    x = df2['worldVidTimeSec'][startIndex:endIndex],
    y = df2['cycGIW_el'][startIndex:endIndex],
    name = 'cycGIW_el',
    marker_color = colors_idx[0],
    mode='markers',
    marker_size = 5,
    opacity = 1)

eih_az = go.Scattergl(
    x = df2['worldVidTimeSec'][startIndex:endIndex],
    y = df2['cycGIW_az'][startIndex:endIndex],
    name = 'cycGIW_az',
    marker_color = colors_idx[1],
    mode='markers',
    marker_size = 5,
    opacity = 1)

traces.append(eih_el)
traces.append(eih_az)

layout = dict(
    dragmode= 'pan',
    title='Time Series with Rangeslider',
    width=width,
    height=height,
    yaxis=dict(range=yLim, title='angular position (degrees)'),
    xaxis=dict(
        rangeslider=dict(visible=True))
)


fig = dict(data=traces, layout=layout)

iplot(fig)


Plotting from index: 1 to 2974
